In [4]:
print("ok")

ok


In [5]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-Ai\\research'

In [6]:
import os 
os.chdir("../")

In [7]:
%pwd

'd:\\End-to-end-Medical-Chatbot-Generative-Ai'

In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [9]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [10]:
extracted_data = load_pdf_files("D:\End-to-end-Medical-Chatbot-Generative-Ai\Data")

In [11]:
#extracted_data

In [12]:
# Split the documents into smaller chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(extracted_data)
    return texts_chunk

In [13]:
texts_chunk = text_split(extracted_data)
print("Length of Text chunks", len(texts_chunk))

Length of Text chunks 7020


In [14]:
#texts_chunk

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

In [16]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\nalba\AppData\Local\Temp\ipykernel_19580\4189599231.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


In [18]:
query_result = embeddings.embed_query("hlo World")
print("Length", len(query_result))

Length 384


In [19]:
#query_result

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
PINECONE_API_KEY =os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY =os.environ.get('OPENAI_API_KEY')

In [22]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",  # Cosine similarity
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)


In [23]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [24]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    index_name=index_name,
    embedding=embeddings
)

ProtocolError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
docsearch


In [ ]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='0915f56b-9056-4562-80ba-57a0c9ce21cc', metadata={'page': 37.0, 'source': 'D:\\End-to-end-Medical-Chatbot-Generative-Ai\\Data\\Medical_book.pdf'}, page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(id='0c6991d2-e4e3-4b68-89a0-b8b234974cc9', metadata={'page': 38.0, 'source': 'D:\\End-to-end-Medical-Chatbot-Generative-Ai\\Data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  P

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.4, max_tokens=500)
#chatModel = ChatOpenAI(model="gpt-4o")

TypeError: cannot pickle 'classmethod' object

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an Medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "what is Acne?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}